In [10]:
import pandas as pd
import numpy as np
import StringIO
import urllib
import random
import datetime
import sys
import time

pd.set_option('max_colwidth', 5000)

HOST = 'http://mtgiga001-1.metrika.yandex.net:8123'
import requests

def get_clickhouse_data(query, wo_cache = False, host = HOST, connection_timeout = 1500):
    if wo_cache:
        r = requests.post(host, data =  query, timeout = connection_timeout,
                     params = {'min_bytes_to_use_direct_io': 1, 'merge_tree_uniform_read_distribution': 0, 'distributed_product_mode': 'local'})
    else:
        r = requests.post(host, data =  query, timeout = connection_timeout,
                         params = {'distributed_product_mode': 'local'})
    
    if r.status_code == 200:
        return r.text
    else:
        raise ValueError, r.text
        
def get_clickhouse_df(query, host = HOST, connection_timeout = 1500):
    data = get_clickhouse_data(query, host, connection_timeout) 
    df = pd.read_csv(StringIO.StringIO(data), sep = '\t')
    return df

def df_to_wiki(df, show_index = False):
    print '#|\n ||' + df.to_csv(index = show_index, sep = '|').replace('|', ' | ').replace('\n', '|| \n ||')[:-2] + '|#'

In [2]:
TABLES = {
    'visits_all': 'default',
    'hits_all': 'default',
    'zgmnkv_counter_key.visits_all': 'zgmnkv_counter_key',
    'zgmnkv_counter_key.hits_all': 'zgmnkv_counter_key',
    'zgmnkv_counter_key_day.visits_all': 'zgmnkv_counter_key_day',
    'zgmnkv_counter_key_day.hits_all': 'zgmnkv_counter_key_day',
    'zgmnkv_month_date_counter.visits_all': 'zgmnkv_month_date_counter',
    'zgmnkv_month_date_counter.hits_all': 'zgmnkv_month_date_counter',
    'zgmnkv_week_counter_date.visits_all': 'zgmnkv_week_counter_date',
    'zgmnkv_week_counter_date.hits_all': 'zgmnkv_week_counter_date',
    'zgmnkv_week_date_counter.visits_all': 'zgmnkv_week_date_counter',
    'zgmnkv_week_date_counter.hits_all': 'zgmnkv_week_date_counter'
}

visit_tables = [
     'zgmnkv_month_date_counter.visits_all',
     'zgmnkv_counter_key.visits_all',
     'zgmnkv_counter_key_day.visits_all',
     'zgmnkv_week_counter_date.visits_all',
     'zgmnkv_week_date_counter.visits_all'
]

hits_tables = [
    'zgmnkv_month_date_counter.hits_all',
    'zgmnkv_counter_key_day.hits_all',
    'zgmnkv_week_counter_date.hits_all',
    'zgmnkv_counter_key.hits_all',
    'zgmnkv_week_date_counter.hits_all',
    'hits_all']


## Query 1

In [3]:
q1_tmpl = '''
SELECT 
    TraficSourceID AS `ym:s:lastTrafficSource`, 
    sum(Sign) AS `ym:s:visits`, 
    least(uniq(UserID), `ym:s:visits`) AS `ym:s:users`, 
    100. * (sum(IsBounce * Sign) / `ym:s:visits`) AS `ym:s:bounceRate`, 
    sum(PageViews * Sign) / `ym:s:visits` AS `ym:s:pageDepth`, 
    sum(Duration * Sign) / `ym:s:visits` AS `ym:s:avgVisitDurationSeconds`, 
    uniqUpToIf(1)((if((RefererDomain != '') AND (`ym:s:lastTrafficSource` IN (-1, toInt8(1))), `ym:s:lastTrafficSource`, if(`ym:s:lastTrafficSource` IN (toInt8(2), toInt8(3), toInt8(8)), `ym:s:lastTrafficSource`, toInt8(0))), if((RefererDomain != '') AND (`ym:s:lastTrafficSource` IN (-1, toInt8(1))), RefererDomain, if(`ym:s:lastTrafficSource` = toInt8(2), toString(toUInt8(dictGetHierarchy('SearchEngine', toUInt64(SearchEngineID))[-1])), if(`ym:s:lastTrafficSource` = toInt8(3), dictGetStringOrDefault('adv_engines_places_names', 'Text', halfMD5(dictGetStringOrDefault('adv_engines_places', 'StrID', (AdvEngineID, toUInt16(PlaceID)), 'ya_undefined')), 'Яндекс: не определено'), if(`ym:s:lastTrafficSource` = toInt8(8), toString(SocialSourceNetworkID), ''))))), (if((RefererDomain != '') AND (`ym:s:lastTrafficSource` IN (-1, toInt8(1))), `ym:s:lastTrafficSource`, if(`ym:s:lastTrafficSource` IN (toInt8(2), toInt8(3), toInt8(8)), `ym:s:lastTrafficSource`, toInt8(0))) != toInt8(0)) OR (if((RefererDomain != '') AND (`ym:s:lastTrafficSource` IN (-1, toInt8(1))), RefererDomain, if(`ym:s:lastTrafficSource` = toInt8(2), toString(toUInt8(dictGetHierarchy('SearchEngine', toUInt64(SearchEngineID))[-1])), if(`ym:s:lastTrafficSource` = toInt8(3), dictGetStringOrDefault('adv_engines_places_names', 'Text', halfMD5(dictGetStringOrDefault('adv_engines_places', 'StrID', (AdvEngineID, toUInt16(PlaceID)), 'ya_undefined')), 'Яндекс: не определено'), if(`ym:s:lastTrafficSource` = toInt8(8), toString(SocialSourceNetworkID), '')))) != '')) AS `ym:s:uniqUpTo1LastSourceEngine`
FROM {table}
WHERE (StartDate >= toDate('{date1}')) AND (StartDate <= toDate('{date2}')) AND (CounterID = 1282253)
GROUP BY `ym:s:lastTrafficSource`
    WITH TOTALS
HAVING (`ym:s:visits` > 0.) OR (`ym:s:users` > 0.)
ORDER BY 
    `ym:s:visits` DESC, 
    `ym:s:lastTrafficSource` ASC
LIMIT 0, 50
'''

In [4]:
import time
def execute_query(query_tmpl, date1, date2, table, query_num, i, wo_cache):
    for n in range(5):
        try:
            time1 = time.time()
            tmp = get_clickhouse_data(
                query_tmpl.format(
                    date1 = date1.strftime('%Y-%m-%d'),
                    date2 = date2.strftime('%Y-%m-%d'),
                    table = table
                ), wo_cache
            )
            time2 = time.time()
        except Exception as e:
            time.sleep(1)
            if n == 4:
                raise e
                return {
                    'date1': date1.strftime('%Y-%m-%d'),
                    'date2': date2.strftime('%Y-%m-%d'),
                    'table': TABLES[table],
                    'timing': 0,
                    'query': query_num, 
                    'iter': i, 
                    'type': ('cache' if not wo_cache else 'wo_cache')
                }
            else:
                print 'Attempt #%d failed' % n
                continue
        else:
            break
        
    
    item = {
        'date1': date1.strftime('%Y-%m-%d'),
        'date2': date2.strftime('%Y-%m-%d'),
        'table': TABLES[table],
        'timing': time2 - time1,
        'query': query_num, 
        'iter': i, 
        'type': ('cache' if not wo_cache else 'wo_cache')
    }
    return item

In [5]:
%%time
q1_stats = []

for date1, date2 in (
    (datetime.datetime(2017, 10, 12), datetime.datetime(2017, 10, 12)),
    (datetime.datetime(2017, 10, 9), datetime.datetime(2017, 10, 22)),
    (datetime.datetime(2017, 10, 1), datetime.datetime(2017, 10, 31)),
):
    print date1, date2
                
    for table in visit_tables:
        print table
        for i in range(3):
            print i,
            q1_stats.append(execute_query(q1_tmpl, date1, date2, table, 1, i, True))
        print
        
        for i in range(5):
            print i,
            q1_stats.append(execute_query(q1_tmpl, date1, date2, table, 1, i, False))
        print

2017-10-12 00:00:00 2017-10-12 00:00:00
zgmnkv_month_date_counter.visits_all
0 1 2
0 1 2 3 4
zgmnkv_counter_key.visits_all
0 1 2
0 1 2 3 4
zgmnkv_counter_key_day.visits_all
0 1 2
0 1 2 3 4
zgmnkv_week_counter_date.visits_all
0 1 2
0 1 2 3 4
zgmnkv_week_date_counter.visits_all
0 1 2
0 1 2 3 4
2017-10-09 00:00:00 2017-10-22 00:00:00
zgmnkv_month_date_counter.visits_all
0 1 2
0 1 2 3 4
zgmnkv_counter_key.visits_all
0 1 2
0 1 2 3 4
zgmnkv_counter_key_day.visits_all
0 1 2
0 1 2 3 4
zgmnkv_week_counter_date.visits_all
0 1 2
0 1 2 3 4
zgmnkv_week_date_counter.visits_all
0 1 2
0 1 2 3 4
2017-10-01 00:00:00 2017-10-31 00:00:00
zgmnkv_month_date_counter.visits_all
0 1 2
0 1 2 3 4
zgmnkv_counter_key.visits_all
0 1 2
0 1 2 3 4
zgmnkv_counter_key_day.visits_all
0 1 2
0 1 2 3 4
zgmnkv_week_counter_date.visits_all
0 1 2
0 1 2 3 4
zgmnkv_week_date_counter.visits_all
0 1 2
0 1 2 3 4
CPU times: user 560 ms, sys: 156 ms, total: 716 ms
Wall time: 3min 22s


In [6]:
def show_stats(stats):
    stats_df = pd.DataFrame(stats)
    wo_cache_stats_df = stats_df[(stats_df.type == 'wo_cache')]
    cache_stats_df = stats_df[(stats_df.type == 'cache') & (stats_df.iter >= 2)]
    full_stats_df = pd.concat([wo_cache_stats_df, cache_stats_df])
    full_stats_df['date_period'] = full_stats_df.date1 + ' - ' + full_stats_df.date2
    full_stats_df['timing'] = map(lambda x: round(x, 3), full_stats_df.timing)
    stats_df_aggr = full_stats_df.groupby(['date_period', 'table', 'type'], as_index = False)[['timing']].median()
    fin_df = stats_df_aggr[stats_df_aggr.type == 'cache'].drop('type', axis = 1).merge(
        stats_df_aggr[stats_df_aggr.type == 'wo_cache'].drop('type', axis = 1),
        on = ['date_period', 'table'],
        suffixes = ('_cache', '_nocache')
    ).applymap(lambda x: x if x != 0 else 'error')
    
    fin_df.sort_values(['date_period', 'table'], ascending = False, inplace = True)
    print '=== 1 день'
    df_to_wiki(fin_df[fin_df.date_period == '2017-10-12 - 2017-10-12'])
    print
    print '=== 2 недели'
    df_to_wiki(fin_df[fin_df.date_period == '2017-10-09 - 2017-10-22'])
    print
    print '=== 1 месяц'
    df_to_wiki(fin_df[fin_df.date_period == '2017-10-01 - 2017-10-31'])

In [7]:
show_stats(q1_stats)

=== 1 день
#|
 ||date_period | table | timing_cache | timing_nocache|| 
 ||2017-10-12 - 2017-10-12 | zgmnkv_week_date_counter | 0.262 | 0.562|| 
 ||2017-10-12 - 2017-10-12 | zgmnkv_week_counter_date | 0.266 | 0.458|| 
 ||2017-10-12 - 2017-10-12 | zgmnkv_month_date_counter | 0.294 | 0.49|| 
 ||2017-10-12 - 2017-10-12 | zgmnkv_counter_key_day | 0.26 | 0.532|| 
 ||2017-10-12 - 2017-10-12 | zgmnkv_counter_key | 0.255 | 0.408|| 
 |#

=== 2 недели
#|
 ||date_period | table | timing_cache | timing_nocache|| 
 ||2017-10-09 - 2017-10-22 | zgmnkv_week_date_counter | 1.127 | 3.464|| 
 ||2017-10-09 - 2017-10-22 | zgmnkv_week_counter_date | 1.153 | 1.361|| 
 ||2017-10-09 - 2017-10-22 | zgmnkv_month_date_counter | 1.062 | 3.684|| 
 ||2017-10-09 - 2017-10-22 | zgmnkv_counter_key_day | 1.145 | 2.162|| 
 ||2017-10-09 - 2017-10-22 | zgmnkv_counter_key | 1.115 | 1.06|| 
 |#

=== 1 месяц
#|
 ||date_period | table | timing_cache | timing_nocache|| 
 ||2017-10-01 - 2017-10-31 | zgmnkv_week_date_counter | 3.

## Query 2

In [8]:
q2_tmpl = '''
SELECT 
    TraficSourceID AS `ym:s:lastTrafficSource`, 
    sum(Sign) AS `ym:s:visits`, 
    least(uniq(UserID), `ym:s:visits`) AS `ym:s:users`, 
    100. * (sum(IsBounce * Sign) / `ym:s:visits`) AS `ym:s:bounceRate`, 
    sum(PageViews * Sign) / `ym:s:visits` AS `ym:s:pageDepth`, 
    sum(Duration * Sign) / `ym:s:visits` AS `ym:s:avgVisitDurationSeconds`, 
    uniqUpToIf(1)((if((RefererDomain != '') AND (`ym:s:lastTrafficSource` IN (-1, toInt8(1))), `ym:s:lastTrafficSource`, if(`ym:s:lastTrafficSource` IN (toInt8(2), toInt8(3), toInt8(8)), `ym:s:lastTrafficSource`, toInt8(0))), if((RefererDomain != '') AND (`ym:s:lastTrafficSource` IN (-1, toInt8(1))), RefererDomain, if(`ym:s:lastTrafficSource` = toInt8(2), toString(toUInt8(dictGetHierarchy('SearchEngine', toUInt64(SearchEngineID))[-1])), if(`ym:s:lastTrafficSource` = toInt8(3), dictGetStringOrDefault('adv_engines_places_names', 'Text', halfMD5(dictGetStringOrDefault('adv_engines_places', 'StrID', (AdvEngineID, toUInt16(PlaceID)), 'ya_undefined')), 'Яндекс: не определено'), if(`ym:s:lastTrafficSource` = toInt8(8), toString(SocialSourceNetworkID), ''))))), (if((RefererDomain != '') AND (`ym:s:lastTrafficSource` IN (-1, toInt8(1))), `ym:s:lastTrafficSource`, if(`ym:s:lastTrafficSource` IN (toInt8(2), toInt8(3), toInt8(8)), `ym:s:lastTrafficSource`, toInt8(0))) != toInt8(0)) OR (if((RefererDomain != '') AND (`ym:s:lastTrafficSource` IN (-1, toInt8(1))), RefererDomain, if(`ym:s:lastTrafficSource` = toInt8(2), toString(toUInt8(dictGetHierarchy('SearchEngine', toUInt64(SearchEngineID))[-1])), if(`ym:s:lastTrafficSource` = toInt8(3), dictGetStringOrDefault('adv_engines_places_names', 'Text', halfMD5(dictGetStringOrDefault('adv_engines_places', 'StrID', (AdvEngineID, toUInt16(PlaceID)), 'ya_undefined')), 'Яндекс: не определено'), if(`ym:s:lastTrafficSource` = toInt8(8), toString(SocialSourceNetworkID), '')))) != '')) AS `ym:s:uniqUpTo1LastSourceEngine`
FROM {table}
WHERE (StartDate >= toDate('{date1}')) AND (StartDate <= toDate('{date2}')) AND (CounterID = 1282253) AND (UserID IN 
(
    SELECT `ym:s:userID` AS `ym:u:userID`
    FROM 
    (
        SELECT 
            UserID AS `ym:s:userID`, 
            argMax(Sex - 1, StartDate) AS `ym:s:gender`
        FROM {table}
        WHERE (StartDate >= toDate('{date1}')) AND (StartDate <= toDate('{date2}')) AND (CounterID = 1282253)
        GROUP BY `ym:s:userID`
            WITH TOTALS
        SETTINGS max_rows_to_group_by = 0
    ) 
    WHERE `ym:s:gender` = toInt16(0)
))
GROUP BY `ym:s:lastTrafficSource`
    WITH TOTALS
HAVING (`ym:s:visits` > 0.) OR (`ym:s:users` > 0.)
ORDER BY 
    `ym:s:visits` DESC, 
    `ym:s:lastTrafficSource` ASC
LIMIT 0, 50
'''

In [11]:
q2_stats = []
for date1, date2 in (
    (datetime.datetime(2017, 10, 12), datetime.datetime(2017, 10, 12)),
    (datetime.datetime(2017, 10, 9), datetime.datetime(2017, 10, 22)),
    (datetime.datetime(2017, 10, 1), datetime.datetime(2017, 10, 31)),
):
    print date1, date2
                
    for table in visit_tables:
        print table
        for i in range(3):
            print i,
            q2_stats.append(execute_query(q2_tmpl, date1, date2, table, 1, i, True))
        print
        
        for i in range(5):
            print i,
            q2_stats.append(execute_query(q2_tmpl, date1, date2, table, 1, i, False))
        print

2017-10-12 00:00:00 2017-10-12 00:00:00
zgmnkv_month_date_counter.visits_all
0 1 2
0 1 2 3 4
zgmnkv_counter_key.visits_all
0 1 2
0 1 2 3 4
zgmnkv_counter_key_day.visits_all
0 1 2
0 1 2 3 4
zgmnkv_week_counter_date.visits_all
0 1 2
0 1 2 3 4
zgmnkv_week_date_counter.visits_all
0 1 2
0 1 2 3 4
2017-10-09 00:00:00 2017-10-22 00:00:00
zgmnkv_month_date_counter.visits_all
0 1 2
0 1 2 3 4
zgmnkv_counter_key.visits_all
0 1 2
0 1 2 3 4
zgmnkv_counter_key_day.visits_all
0 1 2
0 1 2 3 4
zgmnkv_week_counter_date.visits_all
0 1 2
0 1 2 3 4
zgmnkv_week_date_counter.visits_all
0 1 2
0 1 2 3 4
2017-10-01 00:00:00 2017-10-31 00:00:00
zgmnkv_month_date_counter.visits_all
0 1 2
0 1 2 3 4
zgmnkv_counter_key.visits_all
0 1 2
0 1 2 3 4
zgmnkv_counter_key_day.visits_all
0 1 2
0 1 2 3 4
zgmnkv_week_counter_date.visits_all
0 1 2
0 1 2 3 4
zgmnkv_week_date_counter.visits_all
0 1 2
0 1 2 3 4


In [12]:
show_stats(q2_stats)

=== 1 день
#|
 ||date_period | table | timing_cache | timing_nocache|| 
 ||2017-10-12 - 2017-10-12 | zgmnkv_week_date_counter | 0.342 | 0.669|| 
 ||2017-10-12 - 2017-10-12 | zgmnkv_week_counter_date | 0.311 | 0.706|| 
 ||2017-10-12 - 2017-10-12 | zgmnkv_month_date_counter | 0.492 | 0.909|| 
 ||2017-10-12 - 2017-10-12 | zgmnkv_counter_key_day | 0.442 | 1.017|| 
 ||2017-10-12 - 2017-10-12 | zgmnkv_counter_key | 0.476 | 0.983|| 
 |#

=== 2 недели
#|
 ||date_period | table | timing_cache | timing_nocache|| 
 ||2017-10-09 - 2017-10-22 | zgmnkv_week_date_counter | 1.093 | 4.993|| 
 ||2017-10-09 - 2017-10-22 | zgmnkv_week_counter_date | 1.031 | 1.941|| 
 ||2017-10-09 - 2017-10-22 | zgmnkv_month_date_counter | 1.677 | 5.426|| 
 ||2017-10-09 - 2017-10-22 | zgmnkv_counter_key_day | 2.072 | 4.405|| 
 ||2017-10-09 - 2017-10-22 | zgmnkv_counter_key | 1.039 | 1.433|| 
 |#

=== 1 месяц
#|
 ||date_period | table | timing_cache | timing_nocache|| 
 ||2017-10-01 - 2017-10-31 | zgmnkv_week_date_counter |

## Query 3

In [13]:
def execute_query2(query_tmpl, date1, date2, table, query_num, i, wo_cache):
    for n in range(5):
        try:
            time1 = time.time()
            tmp = get_clickhouse_data(
                query_tmpl.format(
                    date1 = date1.strftime('%Y-%m-%d'),
                    date2 = date2.strftime('%Y-%m-%d'),
                    table2 = table,
                    table1 = table.replace('visits', 'hits')
                ), wo_cache
            )
            time2 = time.time()
        except Exception as e:
            time.sleep(1)
            if n == 4:
                return {
                    'date1': date1.strftime('%Y-%m-%d'),
                    'date2': date2.strftime('%Y-%m-%d'),
                    'table': TABLES[table],
                    'timing': 0,
                    'query': query_num, 
                    'iter': i, 
                    'type': ('cache' if not wo_cache else 'wo_cache')
                }
            else:
                print 'Attempt #%d failed' % n
                continue
        else:
            break
        
    
    item = {
        'date1': date1.strftime('%Y-%m-%d'),
        'date2': date2.strftime('%Y-%m-%d'),
        'table': TABLES[table],
        'timing': time2 - time1,
        'query': query_num, 
        'iter': i, 
        'type': ('cache' if not wo_cache else 'wo_cache')
    }
    return item

In [14]:
q3_tmpl = '''
SELECT 
    TraficSourceID AS `ym:s:lastTrafficSource`, 
    sum(Sign) AS `ym:s:visits`, 
    least(uniq(UserID), `ym:s:visits`) AS `ym:s:users`, 
    100. * (sum(IsBounce * Sign) / `ym:s:visits`) AS `ym:s:bounceRate`, 
    sum(PageViews * Sign) / `ym:s:visits` AS `ym:s:pageDepth`, 
    sum(Duration * Sign) / `ym:s:visits` AS `ym:s:avgVisitDurationSeconds`, 
    uniqUpToIf(1)((if((RefererDomain != '') AND (`ym:s:lastTrafficSource` IN (-1, toInt8(1))), `ym:s:lastTrafficSource`, if(`ym:s:lastTrafficSource` IN (toInt8(2), toInt8(3), toInt8(8)), `ym:s:lastTrafficSource`, toInt8(0))), if((RefererDomain != '') AND (`ym:s:lastTrafficSource` IN (-1, toInt8(1))), RefererDomain, if(`ym:s:lastTrafficSource` = toInt8(2), toString(toUInt8(dictGetHierarchy('SearchEngine', toUInt64(SearchEngineID))[-1])), if(`ym:s:lastTrafficSource` = toInt8(3), dictGetStringOrDefault('adv_engines_places_names', 'Text', halfMD5(dictGetStringOrDefault('adv_engines_places', 'StrID', (AdvEngineID, toUInt16(PlaceID)), 'ya_undefined')), 'Яндекс: не определено'), if(`ym:s:lastTrafficSource` = toInt8(8), toString(SocialSourceNetworkID), ''))))), (if((RefererDomain != '') AND (`ym:s:lastTrafficSource` IN (-1, toInt8(1))), `ym:s:lastTrafficSource`, if(`ym:s:lastTrafficSource` IN (toInt8(2), toInt8(3), toInt8(8)), `ym:s:lastTrafficSource`, toInt8(0))) != toInt8(0)) OR (if((RefererDomain != '') AND (`ym:s:lastTrafficSource` IN (-1, toInt8(1))), RefererDomain, if(`ym:s:lastTrafficSource` = toInt8(2), toString(toUInt8(dictGetHierarchy('SearchEngine', toUInt64(SearchEngineID))[-1])), if(`ym:s:lastTrafficSource` = toInt8(3), dictGetStringOrDefault('adv_engines_places_names', 'Text', halfMD5(dictGetStringOrDefault('adv_engines_places', 'StrID', (AdvEngineID, toUInt16(PlaceID)), 'ya_undefined')), 'Яндекс: не определено'), if(`ym:s:lastTrafficSource` = toInt8(8), toString(SocialSourceNetworkID), '')))) != '')) AS `ym:s:uniqUpTo1LastSourceEngine`
FROM {table2}
WHERE (StartDate >= toDate('{date1}')) AND (StartDate <= toDate('{date2}')) AND (CounterID = 1282253) AND arrayExists(x_0 -> (x_0 IN 
(
    SELECT WatchID AS `ym:pv:eventID`
    FROM {table1}
    WHERE (EventDate >= toDate('{date1}')) AND (EventDate <= toDate('{date2}')) AND (CounterID = 1282253) AND NOT DontCountHits AND NOT Refresh AND (positionCaseInsensitive(URL, 'woman.ru') > toInt32(0))
)), `Event.ID`)
GROUP BY `ym:s:lastTrafficSource`
    WITH TOTALS
HAVING (`ym:s:visits` > 0.) OR (`ym:s:users` > 0.)
ORDER BY 
    `ym:s:visits` DESC, 
    `ym:s:lastTrafficSource` ASC
LIMIT 0, 50
'''

In [16]:
q3_stats = []
for date1, date2 in (
    (datetime.datetime(2017, 10, 12), datetime.datetime(2017, 10, 12)),
    (datetime.datetime(2017, 10, 9), datetime.datetime(2017, 10, 22)),
    (datetime.datetime(2017, 10, 1), datetime.datetime(2017, 10, 31)),
):
    print date1, date2
                
    for table in visit_tables:
        print table
        for i in range(3):
            print i,
            q3_stats.append(execute_query2(q3_tmpl, date1, date2, table, 1, i, True))
        print
        
        for i in range(5):
            print i,
            q3_stats.append(execute_query2(q3_tmpl, date1, date2, table, 1, i, False))
        print

2017-10-12 00:00:00 2017-10-12 00:00:00
zgmnkv_month_date_counter.visits_all
0 1 2
0 1 2 3 4
zgmnkv_counter_key.visits_all
0 1 2
0 1 2 Attempt #0 failed
3 4
zgmnkv_counter_key_day.visits_all
0 1 2
0 1 2 3 4
zgmnkv_week_counter_date.visits_all
0 1 2
0 1 2 3 4
zgmnkv_week_date_counter.visits_all
0 1 2
0 1 2 3 4
2017-10-09 00:00:00 2017-10-22 00:00:00
zgmnkv_month_date_counter.visits_all
0 1 2
0 1 2 3 4
zgmnkv_counter_key.visits_all
0 1 2
0 1 2 3 4
zgmnkv_counter_key_day.visits_all
0 1 2
0 1 2 3 4
zgmnkv_week_counter_date.visits_all
0 1 2
0 1 2 3 4
zgmnkv_week_date_counter.visits_all
0 1 2
0 1 2 3 4
2017-10-01 00:00:00 2017-10-31 00:00:00
zgmnkv_month_date_counter.visits_all
0 1 2
0 1 2 3 4
zgmnkv_counter_key.visits_all
0 1 2
0 1 2 3 4
zgmnkv_counter_key_day.visits_all
0 1 2
0 1 2 3 4
zgmnkv_week_counter_date.visits_all
0 1 2
0 1 2 3 4
zgmnkv_week_date_counter.visits_all
0 1 2
0 1 2 3 4


In [17]:
show_stats(q3_stats)

=== 1 день
#|
 ||date_period | table | timing_cache | timing_nocache|| 
 ||2017-10-12 - 2017-10-12 | zgmnkv_week_date_counter | 0.983 | 1.646|| 
 ||2017-10-12 - 2017-10-12 | zgmnkv_week_counter_date | 1.012 | 1.729|| 
 ||2017-10-12 - 2017-10-12 | zgmnkv_month_date_counter | 0.37 | 1.048|| 
 ||2017-10-12 - 2017-10-12 | zgmnkv_counter_key_day | 0.678 | 0.896|| 
 ||2017-10-12 - 2017-10-12 | zgmnkv_counter_key | 0.558 | 0.934|| 
 |#

=== 2 недели
#|
 ||date_period | table | timing_cache | timing_nocache|| 
 ||2017-10-09 - 2017-10-22 | zgmnkv_week_date_counter | 3.618 | 7.178|| 
 ||2017-10-09 - 2017-10-22 | zgmnkv_week_counter_date | 2.466 | 4.306|| 
 ||2017-10-09 - 2017-10-22 | zgmnkv_month_date_counter | 2.352 | 8.49|| 
 ||2017-10-09 - 2017-10-22 | zgmnkv_counter_key_day | 2.042 | 4.331|| 
 ||2017-10-09 - 2017-10-22 | zgmnkv_counter_key | 1.083 | 2.084|| 
 |#

=== 1 месяц
#|
 ||date_period | table | timing_cache | timing_nocache|| 
 ||2017-10-01 - 2017-10-31 | zgmnkv_week_date_counter | 7